# Binary Classification of Insurance Selling - Model Selection

The aim of this workbook is to build on the previous workbook in order to improve the roc_auc score achieved.

url: https://www.kaggle.com/competitions/playground-series-s4e7/overview

I intend to train multiple different models using the same preprocessing and upsampling techniques outlined in the first workbook.